In [ ]:
# 환경 변수 로드 및 자동 리로드 설정
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

# 사용자 명확화 및 브리프 생성

*스코핑의 목표는 연구에 필요한 사용자 컨텍스트를 수집하는 것입니다.*

다음은 우리의 전체 연구 플로우입니다:

![image.png](attachment:6b1de3cc-2d28-48ed-8f02-56b4a0a749e0.png)

연구를 두 단계로 스코핑합니다:

1. **사용자 명확화** - 사용자로부터 추가 명확화가 필요한지 결정
2. **브리프 생성** - 대화를 상세한 연구 브리프로 변환

![image.webp](attachment:572e3be3-d2fb-41a3-af44-351baca997c0.webp)

### 프롬프트

심층 연구 워크플로우에서 흔한 문제는 사용자가 초기 요청에서 충분한 컨텍스트를 거의 제공하지 않는다는 것입니다.

요청에는 종종 다음과 같은 중요한 세부사항이 부족합니다:

- **범위와 경계**: 무엇을 포함하거나 제외해야 하는가?
- **대상과 목적**: 이 연구는 누구를 위한 것이고 왜 필요한가?
- **특정 요구사항**: 특정 소스, 시간대 또는 제약사항이 있는가?
- **용어 명확화**: 도메인별 용어나 약어는 무엇을 의미하는가?

가정을 하기보다는 목표가 있는 명확화 질문을 통해 추가 컨텍스트를 수집합니다.

이를 통해 목표를 벗어날 수 있는 연구에 시간을 투자하기 전에 사용자의 진정한 의도를 이해할 수 있습니다.

![image (1).webp](attachment:9f6f7dd6-215b-4183-8b85-3ccb5586f6ee.webp)

In [ ]:
from utils import show_prompt
from deep_research_from_scratch.prompts import clarify_with_user_instructions
show_prompt(clarify_with_user_instructions, "사용자 명확화 지침")

### 상태 및 스키마

먼저 연구 프로세스를 위한 상태 객체와 스키마를 정의합니다.

상태 객체는 연구 워크플로우의 다양한 단계 간에 컨텍스트를 저장하고 전달하는 주요 메커니즘 역할을 합니다.

이를 사용하여 연구를 안내하는 데 사용될 [컨텍스트를 작성하고 선택](https://blog.langchain.com/context-engineering-for-agents/)할 수 있습니다.
 
> **참고:** `%%writefile`을 사용하여 코드 블록을 특정 파일 `state_scope.py`에 저장합니다. 이를 통해 향후 노트북에서 쉽게 재사용할 수 있으며, 배포 가능한 LangGraph 애플리케이션에서 직접 사용할 수 있는 코드를 생성합니다!

In [ ]:
%%writefile ../src/deep_research_from_scratch/state_scope.py

"""연구 스코핑을 위한 상태 정의 및 Pydantic 스키마.

연구 에이전트 스코핑 워크플로우에 사용되는 상태 객체와 구조화된 스키마를
정의합니다. 연구자 상태 관리 및 출력 스키마를 포함합니다.
"""

import operator
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

# ===== 상태 정의 =====

class AgentInputState(MessagesState):
    """전체 에이전트의 입력 상태 - 사용자 입력의 메시지만 포함."""
    pass

class AgentState(MessagesState):
    """
    전체 멀티 에이전트 연구 시스템의 메인 상태.
    
    연구 조정을 위한 추가 필드로 MessagesState를 확장합니다.
    참고: 일부 필드는 서브그래프와 메인 워크플로우 간의 적절한
    상태 관리를 위해 다른 상태 클래스에서 중복됩니다.
    """

    # 사용자 대화 기록에서 생성된 연구 브리프
    research_brief: Optional[str]
    # 조정을 위해 슈퍼바이저 에이전트와 교환된 메시지
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages]
    # 연구 단계에서 수집된 원시 미처리 연구 노트
    raw_notes: Annotated[list[str], operator.add] = []
    # 보고서 생성을 위해 준비된 처리되고 구조화된 노트
    notes: Annotated[list[str], operator.add] = []
    # 최종 형식화된 연구 보고서
    final_report: str

# ===== 구조화된 출력 스키마 =====

class ClarifyWithUser(BaseModel):
    """사용자 명확화 결정 및 질문을 위한 스키마."""
    
    need_clarification: bool = Field(
        description="사용자에게 명확화 질문을 해야 하는지 여부.",
    )
    question: str = Field(
        description="보고서 범위를 명확히 하기 위해 사용자에게 할 질문",
    )
    verification: str = Field(
        description="사용자가 필요한 정보를 제공한 후 연구를 시작할 것이라는 확인 메시지.",
    )

class ResearchQuestion(BaseModel):
    """구조화된 연구 브리프 생성을 위한 스키마."""
    
    research_brief: str = Field(
        description="연구를 안내하는 데 사용될 연구 질문.",
    )

### 연구 스코핑

이제 사용자의 의도를 명확히 하고 연구 브리프를 작성하는 간단한 워크플로우를 만들어보겠습니다.

LLM이 브리프를 작성하기에 충분한 명확화가 있는지 결정하도록 하겠습니다.
 
이는 LangGraph의 [Command](https://langchain-ai.github.io/langgraph/how-tos/graph-api/#combine-control-flow-and-state-updates-with-command)를 사용하여 제어 흐름을 지시하고 상태를 업데이트합니다. `Command` 객체는 두 가지 주요 매개변수를 받습니다:
- `goto`: 실행할 다음 노드를 지정 (또는 종료하려면 `END`)
- `update`: 전환하기 전에 적용할 상태 업데이트 딕셔너리

이 패턴을 통해 함수가 데이터를 처리하고 결과에 따라 워크플로우를 지시할 수 있습니다.

기존의 정적 그래프 구조보다 더 유연하고 유지보수 가능한 시스템을 만듭니다.

In [ ]:
%%writefile ../src/deep_research_from_scratch/research_agent_scope.py

"""사용자 명확화 및 연구 브리프 생성.

이 모듈은 연구 워크플로우의 스코핑 단계를 구현합니다:
1. 사용자의 요청에 명확화가 필요한지 평가
2. 대화에서 상세한 연구 브리프 생성

워크플로우는 구조화된 출력을 사용하여 연구를 진행하기에
충분한 컨텍스트가 있는지에 대한 결정론적 결정을 내립니다.
"""

from datetime import datetime
from typing_extensions import Literal

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from deep_research_from_scratch.prompts import clarify_with_user_instructions, transform_messages_into_research_topic_prompt
from deep_research_from_scratch.state_scope import AgentState, ClarifyWithUser, ResearchQuestion, AgentInputState

# ===== 유틸리티 함수 =====

def get_today_str() -> str:
    """현재 날짜를 사람이 읽기 쉬운 형식으로 가져옵니다."""
    return datetime.now().strftime("%a %b %-d, %Y")

# ===== 설정 =====

# 모델 초기화
model = init_chat_model(model="openai:gpt-4.1", temperature=0.0)

# ===== 워크플로우 노드 =====

def clarify_with_user(state: AgentState) -> Command[Literal["write_research_brief", "__end__"]]:
    """
    사용자의 요청에 연구를 진행하기에 충분한 정보가 포함되어 있는지 결정합니다.
    
    구조화된 출력을 사용하여 결정론적 결정을 내리고 환각을 방지합니다.
    연구 브리프 생성 또는 명확화 질문으로 종료하도록 라우팅합니다.
    """
    # 구조화된 출력 모델 설정
    structured_output_model = model.with_structured_output(ClarifyWithUser)

    # 명확화 지침으로 모델 호출
    response = structured_output_model.invoke([
        HumanMessage(content=clarify_with_user_instructions.format(
            messages=get_buffer_string(messages=state["messages"]), 
            date=get_today_str()
        ))
    ])
    
    # 명확화 필요성에 따른 라우팅
    if response.need_clarification:
        return Command(
            goto=END, 
            update={"messages": [AIMessage(content=response.question)]}
        )
    else:
        return Command(
            goto="write_research_brief", 
            update={"messages": [AIMessage(content=response.verification)]}
        )

def write_research_brief(state: AgentState):
    """
    대화 기록을 포괄적인 연구 브리프로 변환합니다.
    
    구조화된 출력을 사용하여 브리프가 필요한 형식을 따르고
    효과적인 연구에 필요한 모든 세부사항을 포함하도록 합니다.
    """
    # 구조화된 출력 모델 설정
    structured_output_model = model.with_structured_output(ResearchQuestion)
    
    # 대화 기록에서 연구 브리프 생성
    response = structured_output_model.invoke([
        HumanMessage(content=transform_messages_into_research_topic_prompt.format(
            messages=get_buffer_string(state.get("messages", [])),
            date=get_today_str()
        ))
    ])
    
    # 생성된 연구 브리프로 상태 업데이트하고 슈퍼바이저에게 전달
    return {
        "research_brief": response.research_brief,
        "supervisor_messages": [HumanMessage(content=f"{response.research_brief}.")]
    }

# ===== 그래프 구성 =====

# 스코핑 워크플로우 구축
deep_researcher_builder = StateGraph(AgentState, input_schema=AgentInputState)

# 워크플로우 노드 추가
deep_researcher_builder.add_node("clarify_with_user", clarify_with_user)
deep_researcher_builder.add_node("write_research_brief", write_research_brief)

# 워크플로우 엣지 추가
deep_researcher_builder.add_edge(START, "clarify_with_user")
deep_researcher_builder.add_edge("write_research_brief", END)

# 워크플로우 컴파일
scope_research = deep_researcher_builder.compile()

In [ ]:
# 노트북에서 테스트하기 위해 인메모리 체크포인터로 컴파일
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from deep_research_from_scratch.research_agent_scope import deep_researcher_builder

checkpointer = InMemorySaver()
scope = deep_researcher_builder.compile(checkpointer=checkpointer)
display(Image(scope.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 워크플로우 실행
from utils import format_messages
from langchain_core.messages import HumanMessage
thread = {"configurable": {"thread_id": "1"}}
result = scope.invoke({"messages": [HumanMessage(content="샌프란시스코 최고의 커피숍을 연구하고 싶습니다.")]}, config=thread)
format_messages(result['messages'])

In [ ]:
result = scope.invoke({"messages": [HumanMessage(content="샌프란시스코 최고의 커피숍을 평가하기 위해 커피 품질을 살펴보겠습니다.")]}, config=thread)
format_messages(result['messages'])

In [ ]:
from rich.markdown import Markdown
Markdown(result["research_brief"])

[추적](https://smith.langchain.com/public/75278fdf-4468-4dcc-bf44-a28ab6018d92/r)을 확인할 수 있습니다.

### 로컬 배포 및 LangGraph Studio

LangGraph는 배포를 원활하게 지원하도록 설계되었습니다.

이 노트북에서 `%%writefile`로 `src/deep_research_from_scratch/`에 작성한 파일들이 애플리케이션의 기반을 형성합니다:

```
deep_research_from_scratch/
├── src/deep_research_from_scratch/
│   ├── state.py          # 상태 정의
│   ├── scope_research.py # 스코핑 워크플로우
│   ├── prompts.py        # 프롬프트 템플릿
│   └── ...
├── notebooks/            # 개발 노트북
├── pyproject.toml        # 의존성
└── langgraph.json        # LangGraph 설정
```

또한 저장소에는 의존성, 그래프 및 환경 변수를 지정하는 `langgraph.json` 파일이 있습니다.

이 구조를 통해 스코핑 워크플로우를 [로컬 또는 원격 서버를 통해 배포](https://langchain-ai.github.io/langgraph/concepts/deployment_options/)할 수 있습니다.

저장소 루트에서 아래 명령으로 LangGraph 서버를 로컬에서 시작할 수 있으며, 이는 브라우저에서 LangGraph Studio를 열어줍니다:

```bash
uvx --refresh --from "langgraph-cli[inmem]" --with-editable . --python 3.11 langgraph dev
```

### 평가

이제 연구를 스코핑했으므로 몇 가지 예제로 테스트하여 예상대로 작동하는지 확인해보겠습니다.

좋은 연구 브리프가 무엇인지 생각해봅시다:

* 사용자 채팅에서 관련 기준을 포착합니다
* 사용자가 명시적으로 제공하지 않은 기준을 발명하거나 가정하지 않습니다

다음은 두 가지 샘플 입력 대화입니다.

In [ ]:
from langchain_core.messages import AIMessage

conversation_1 = [
    HumanMessage(content="은퇴를 위해 5만 달러를 투자하는 가장 좋은 방법은 무엇인가요?"),
    AIMessage(content="5만 달러 은퇴 목표에 맞는 투자 조언을 제공하기 위해 추가 정보를 제공해 주시겠습니까? 구체적으로:\n 현재 나이 또는 원하는 은퇴 나이\n 위험 허용도 (낮음, 중간, 높음)\n 투자 유형에 대한 선호도 (예: 주식, 채권, 뮤추얼 펀드, 부동산)\n 세금 우대 계좌 (예: IRA, 401(k)) 또는 일반 중개 계좌를 통해 투자하는지 여부\n 이를 통해 더 개인화되고 관련성 있는 제안을 제공할 수 있습니다."),
    HumanMessage(content="저는 25세이고 45세까지 은퇴하고 싶습니다. 현재 위험 허용도는 높지만 시간이 지나면서 감소할 것 같습니다. 주식과 ETF가 좋은 선택이라고 들었지만 무엇이든 열려있습니다. 그리고 이미 401k가 있지만 이것은 일반 중개 계좌를 통할 것입니다."),
]

conversation_2 = [
    HumanMessage(content="뉴욕에서 아파트를 찾고 있는데 도와주실 수 있나요?"),
    AIMessage(content="아파트 선호도를 구체적으로 알려주시겠습니까? 예를 들어:\n 원하는 동네나 자치구\n 침실/욕실 수\n 예산 범위 (월 임대료)\n 편의시설이나 필수 기능\n 선호하는 입주 날짜\n 이 정보는 뉴욕에서 가장 관련성 있는 아파트 옵션을 제공하는 데 도움이 됩니다."),
    HumanMessage(content="첼시, 플래티론 또는 웨스트 빌리지에 살고 싶습니다. 침실 2개, 욕실 2개를 찾고 있으며 월 임대료는 7천 달러 미만을 원합니다. 도어맨 빌딩이었으면 좋겠고 세탁기와 건조기가 있으면 좋지만 없어도 괜찮습니다. 빌딩에 헬스장이 있으면 플러스입니다. 그리고 2025년 9월에 입주하고 싶습니다."),
]

이제 연구 브리프에서 보존되기를 원하는 각 대화의 기준을 수동으로 작성해보겠습니다.

In [ ]:
criteria_1 = [
    "현재 나이는 25세",
    "원하는 은퇴 나이는 45세",
    "현재 위험 허용도는 높음",
    "주식과 ETF 투자에 관심",
    "주식과 ETF 외의 투자 형태에도 열려있음",
    "투자 계좌는 일반 중개 계좌",
]

criteria_2 = [
    "첼시, 플래티론 또는 웨스트 빌리지에서 침실 2개, 욕실 2개 아파트 찾기",
    "월 임대료 7천 달러 미만",
    "도어맨 빌딩이어야 함",
    "이상적으로는 세탁기와 건조기가 있지만 엄격하지 않음",
    "이상적으로는 헬스장이 있지만 엄격하지 않음",
    "입주 날짜는 2025년 9월"
]

LangSmith를 사용하여 이 실험을 실행하겠습니다.

> 참고: `README`의 관련 지침에 따라 LangSmith 계정을 설정하고 (무료 티어로 충분합니다!) 환경에 `LANGSMITH_API_KEY`를 설정했는지 확인하세요.

LangSmith에서 실험을 실행하는 것은 세 단계로 구성됩니다:

1. 데이터셋 생성
2. 평가자 작성
3. 실험 실행

플로우는 다음과 같습니다:

![Screenshot 2025-07-29 at 3.31.55 PM.png](attachment:838202f9-a45d-44c0-9af8-b824029376a8.png)

데이터셋을 생성하고 두 예제를 추가하는 것부터 시작하겠습니다.

In [ ]:
import os
from langsmith import Client

# LangSmith 클라이언트 초기화
langsmith_client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))

# 데이터셋 생성
dataset_name = "deep_research_scoping"
if not langsmith_client.has_dataset(dataset_name=dataset_name):
    
    # 데이터셋 생성
    dataset = langsmith_client.create_dataset(
        dataset_name=dataset_name,
        description="입력 대화에서 생성된 연구 브리프의 품질을 측정하는 데이터셋",
    )

    # 데이터셋에 예제 추가
    langsmith_client.create_examples(
        dataset_id=dataset.id,
        examples=[
            {
                "inputs": {"messages": conversation_1},
                "outputs": {"criteria": criteria_1},
            },
            {
                "inputs": {"messages": conversation_2},
                "outputs": {"criteria": criteria_2},
            },
        ],
    )

이제 각 예제에 대해 지정한 성공 기준과 연구 브리프를 비교할 평가자를 작성해야 합니다. 이를 위해 LLM-as-judge를 사용하겠습니다. llm-as-judge 평가자 작성에 대한 유용한 팁은 [여기](https://hamel.dev/blog/posts/llm-judge/index.html)에서 찾을 수 있으며, 다음을 포함합니다:

1. **전문성 컨텍스트를 가진 역할 정의**
   - 특정 전문가 역할 정의 ("연구 브리프 평가자", "세심한 감사자")
   - 특정 평가 도메인에 역할 특화

2. **명확한 작업 사양**
   - 이진 통과/실패 판단 (복잡한 다차원 점수 매기기 방지)
   - 명시적 작업 경계 및 목표
   - 실행 가능한 평가 기준에 집중

3. **풍부한 컨텍스트 배경**
   - 연구 브리프 품질에 대한 도메인별 컨텍스트 제공
   - 정확한 평가의 중요성 설명
   - 평가 결과를 다운스트림 결과와 연결

4. **구조화된 XML 조직**
   - 다양한 섹션에 의미론적 XML 태그 사용
   - 역할, 작업, 컨텍스트, 입력, 가이드라인 및 출력의 명확한 분리
   - 향상된 프롬프트 파싱 및 이해

5. **예제가 포함된 포괄적 가이드라인**
   - 특정 조건이 있는 상세한 통과/실패 기준
   - 다양한 시나리오를 다루는 여러 구체적 예제
   - 프롬프트당 3-4개 예제로 다양한 시나리오 커버
   - 각 판단 유형에 대한 긍정적 및 부정적 예제
   - 엣지 케이스 처리 및 결정 경계 명확화

6. **명시적 출력 지침**
   - 평가 기준 적용 방법에 대한 명확한 지침
   - 모호한 경우 처리 지침
   - 일관성과 체계적 평가 강조

7. **편향 감소 기법**
   - 정밀도와 재현율의 균형을 위한 "엄격하지만 공정한" 지침
   - 보수적 평가를 위한 "의심스러울 때는 실패로 기울기"
   - 주관적 변동을 줄이기 위한 체계적 평가 프로세스

In [ ]:
from deep_research_from_scratch.prompts import BRIEF_CRITERIA_PROMPT
show_prompt(BRIEF_CRITERIA_PROMPT, "BRIEF_CRITERIA_PROMPT")

In [ ]:
from typing_extensions import cast
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class Criteria(BaseModel):
    """
    개별 성공 기준 평가 결과.
    
    이 모델은 연구 브리프에 있어야 하는 단일 평가 기준을 나타내며,
    성공적으로 포착되었는지에 대한 상세한 평가와 그 평가 뒤의 추론을 포함합니다.
    """
    criteria_text: str = Field(
        description="평가되는 특정 성공 기준 (예: '현재 나이는 25세', '월 임대료 7천 달러 미만')"
    )
    reasoning: str = Field(
        description="이 기준이 연구 브리프에서 포착되었는지 또는 그렇지 않은지에 대한 상세한 설명, 브리프의 구체적 증거 포함"
    )
    is_captured: bool = Field(
        description="이 특정 기준이 연구 브리프에서 적절히 포착되었는지 (True) 또는 누락/부적절하게 다뤄졌는지 (False)"
    )

def evaluate_success_criteria(outputs: dict, reference_outputs: dict):
    """
    연구 브리프가 모든 필요한 성공 기준을 포착하는지 평가합니다.
    
    이 함수는 각 기준을 개별적으로 평가하여 집중적인 평가와
    각 평가 결정에 대한 상세한 추론을 제공합니다.
    
    Args:
        outputs: 평가할 연구 브리프를 포함하는 딕셔너리
        reference_outputs: 성공 기준 목록을 포함하는 딕셔너리
        
    Returns:
        점수 (0.0에서 1.0)를 포함한 평가 결과 딕셔너리
    """
    research_brief = outputs["research_brief"]
    success_criteria = reference_outputs["criteria"]

    model = ChatOpenAI(model="gpt-4.1", temperature=0)
    structured_output_model = model.with_structured_output(Criteria)
    
    # 평가 실행
    responses = structured_output_model.batch([
    [
        HumanMessage(
            content=BRIEF_CRITERIA_PROMPT.format(
                research_brief=research_brief,
                criterion=criterion
            )
        )
    ] 
    for criterion in success_criteria])
    
    # criteria_text 필드가 올바르게 채워지도록 보장
    individual_evaluations = [
        Criteria(
            reasoning=response.reasoning,
            criteria_text=criterion,
            is_captured=response.is_captured
        )
        for criterion, response in zip(success_criteria, responses)
    ]
    
    # 포착된 기준의 백분율로 전체 점수 계산
    captured_count = sum(1 for eval_result in individual_evaluations if eval_result.is_captured)
    total_count = len(individual_evaluations)
    
    return {
        "key": "success_criteria_score", 
        "score": captured_count / total_count if total_count > 0 else 0.0,
        "individual_evaluations": [
            {
                "criteria": eval_result.criteria_text,
                "captured": eval_result.is_captured,
                "reasoning": eval_result.reasoning
            }
            for eval_result in individual_evaluations
        ]
    }

두 번째 평가자는 연구 브리프가 사용자가 연구 브리프에서 지정하지 않은 가정을 하지 않는지 확인합니다.

In [ ]:
from deep_research_from_scratch.prompts import BRIEF_HALLUCINATION_PROMPT
show_prompt(BRIEF_HALLUCINATION_PROMPT, "BRIEF_HALLUCINATION_PROMPT")

In [ ]:
# 추론 필드와 향상된 설명이 있는 개선된 NoAssumptions 클래스
class NoAssumptions(BaseModel):
    """
    연구 브리프가 부당한 가정을 하는지 확인하는 평가 모델.
    
    이 모델은 연구 브리프에 가정, 추론 또는 사용자가 원래 대화에서
    명시적으로 언급하지 않은 추가 사항이 포함되어 있는지 평가합니다.
    평가 결정에 대한 상세한 추론을 제공합니다.
    """
    no_assumptions: bool = Field(
        description="연구 브리프가 부당한 가정을 피하는지 여부. 브리프가 사용자가 명시적으로 제공한 정보만 포함하면 True, 언급되지 않은 것 이상의 가정을 하면 False."
    )
    reasoning: str = Field(
        description="평가 결정에 대한 상세한 설명, 발견된 가정의 구체적 예제 또는 사용자의 명시적 언급 이상의 가정이 없다는 확인 포함."
    )

def evaluate_no_assumptions(outputs: dict, reference_outputs: dict):
    """
    연구 브리프가 부당한 가정을 피하는지 평가합니다.
    
    이 평가자는 연구 브리프가 사용자가 명시적으로 제공한 정보와
    요구사항만 포함하고, 언급되지 않은 선호도나 요구사항에 대한
    가정을 하지 않는지 확인합니다.
    
    Args:
        outputs: 평가할 연구 브리프를 포함하는 딕셔너리
        reference_outputs: 참조용 성공 기준을 포함하는 딕셔너리
        
    Returns:
        불린 점수와 상세한 추론을 포함한 평가 결과 딕셔너리
    """
    research_brief = outputs["research_brief"]
    success_criteria = reference_outputs["criteria"]
    
    model = ChatOpenAI(model="gpt-4.1", temperature=0)
    structured_output_model = model.with_structured_output(NoAssumptions)
    
    response = structured_output_model.invoke([
        HumanMessage(content=BRIEF_HALLUCINATION_PROMPT.format(
            research_brief=research_brief, 
            success_criteria=str(success_criteria)
        ))
    ])
    
    return {
        "key": "no_assumptions_score", 
        "score": response.no_assumptions,
        "reasoning": response.reasoning
    }

이제 평가자가 있으므로 실험을 실행할 수 있습니다.

In [ ]:
import uuid

def target_func(inputs: dict):
    config = {"configurable": {"thread_id": uuid.uuid4()}}
    return scope.invoke(inputs, config=config)

langsmith_client.evaluate(
    target_func,
    data=dataset_name,
    evaluators=[evaluate_success_criteria, evaluate_no_assumptions],
    experiment_prefix="Deep Research Scoping",
)

위 링크를 클릭하여 결과를 확인할 수 있습니다!

왜 이런 평가를 수행하나요?

* 애플리케이션의 개별 단계가 예상대로 수행되는지 확인합니다.
* LangSmith로의 추적을 통해 각 단계가 얼마나 걸리는지와 비용도 파악할 수 있습니다.
* 더 저렴하고 빠른 모델을 시도해서 작업을 수행할 수 있는지 항상 확인할 수 있습니다.

에이전트가 실수를 한다면 에이전트에게 제공되는 프롬프트를 조정하여 성능을 향상시킬 수도 있습니다!